In [1]:
from dotenv import load_dotenv
import os
from IPython.display import display, Markdown

load_dotenv()
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

In [2]:
openai_api_key = os.environ["OPENAI_API_KEY"]

In [3]:
from langchain_openai import OpenAI

llm = OpenAI(api_key=openai_api_key, temperature=0.9)

In [5]:
text = "What would be a good company name for a company that makes colorful socks?"

print(llm.invoke(text))



"RainbowToes" or "VividSocks"


### OpenAI callback

Source: [Link](https://python.langchain.com/docs/modules/model_io/chat/token_usage_tracking/)

In [6]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4-turbo", api_key=openai_api_key, temperature=0.0)

In [7]:
msg = llm.invoke([("human", "What's the oldest known example of cuneiform")])
msg.response_metadata

{'token_usage': {'completion_tokens': 138,
  'prompt_tokens': 17,
  'total_tokens': 155},
 'model_name': 'gpt-4-turbo',
 'system_fingerprint': 'fp_76f018034d',
 'finish_reason': 'stop',
 'logprobs': None}

In [8]:
from langchain_community.callbacks.manager import get_openai_callback

In [10]:
with get_openai_callback() as cb:
    result = llm.invoke("Tell me about the earth in about a 1000 words.")
    print(cb)

Tokens Used: 1006
	Prompt Tokens: 20
	Completion Tokens: 986
Successful Requests: 1
Total Cost (USD): $0.0


### Few Shot Prompting

Source: [Link](https://python.langchain.com/docs/modules/model_io/prompts/few_shot_examples/)

In [12]:
from langchain_core.prompts.few_shot import FewShotPromptTemplate
from langchain_core.prompts.prompt import PromptTemplate

In [13]:
# create our examples
examples = [
    {
        "query": "What's the weather like?",
        "answer": "It's raining cats and dogs, better bring an umbrella!"
    }, {
        "query": "How old are you?",
        "answer": "Age is just a number, but I'm timeless."
    }
]

# create an example template
example_template = """
User: {query}
AI: {answer}
"""

In [16]:
example_prompt = PromptTemplate(
    input_variables=["query", "answer"],
    template=example_template
)

In [17]:
example_prompt.format(**examples[0])

"\nUser: What's the weather like?\nAI: It's raining cats and dogs, better bring an umbrella!\n"

In [14]:
# now break our previous prompt into a prefix and suffix
# the prefix is our instructions
prefix = """The following are excerpts from conversations with an AI
assistant. The assistant is known for its humor and wit, providing
entertaining and amusing responses to users' questions. Here are some
examples:
"""
# and the suffix our user input and output indicator
suffix = """
User: {query}
AI: """

In [18]:
prompt = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    prefix=prefix,
    suffix=suffix,
    input_variables=["query"],
    example_separator="\n\n"
)


In [22]:
display(Markdown(prompt.format(query="What's the meaning of life?")))

The following are excerpts from conversations with an AI
assistant. The assistant is known for its humor and wit, providing
entertaining and amusing responses to users' questions. Here are some
examples:



User: What's the weather like?
AI: It's raining cats and dogs, better bring an umbrella!



User: How old are you?
AI: Age is just a number, but I'm timeless.



User: What's the meaning of life?
AI: 

In [27]:
llm = OpenAI(api_key=openai_api_key, temperature=0.0)

In [28]:
chain = prompt | llm

In [30]:
display(Markdown(chain.invoke({"query":"What's the meaning of life?"})))

42, according to some famous book. But personally, I think it's just to enjoy the ride and eat lots of pizza.

In [31]:
display(Markdown(chain.invoke({"query":"Why do we exist ?"})))

That's a deep question, let me consult my existential database... It seems we exist to ponder the meaning of our existence. How meta.

In [32]:
display(Markdown(chain.invoke({"query":"Why are people always sad ?"})))

Well, life can be a rollercoaster of emotions. But don't worry, I'm here to cheer you up! Want to hear a joke? 

### Few Shot Prompting of Chat models

Source: [Link](https://python.langchain.com/docs/modules/model_io/prompts/few_shot_examples_chat/)

In [33]:
from langchain_core.prompts import (
    ChatPromptTemplate,
    FewShotChatMessagePromptTemplate,
)

In [34]:
# create our examples
examples = [
    {
        "query": "What's the weather like?",
        "answer": "It's raining cats and dogs, better bring an umbrella!"
    }, {
        "query": "How old are you?",
        "answer": "Age is just a number, but I'm timeless."
    }
]

# create an example template
example_template = """
User: {query}
AI: {answer}
"""

In [43]:
example_prompt = ChatPromptTemplate.from_messages(
        [
        ("user", "{query}"),
        ("ai", "{answer}"),
    ]
)

In [44]:
few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)

print(few_shot_prompt.format())

Human: What's the weather like?
AI: It's raining cats and dogs, better bring an umbrella!
Human: How old are you?
AI: Age is just a number, but I'm timeless.


In [53]:
# now break our previous prompt into a prefix and suffix
# the prefix is our instructions
prefix = """The following are excerpts from conversations with an AI
assistant. The assistant is known for its humor and wit, providing
entertaining and amusing responses to users' questions. Here are some
examples:
"""
# and the suffix our user input and output indicator
suffix = """
User: {query}
AI: """

In [57]:
final_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", prefix),
        few_shot_prompt,
        ("human", "{input}"),
        ("ai", "")
    ]
)

In [58]:
display(Markdown(final_prompt.format(input = "What is the meaning of life ?")))

System: The following are excerpts from conversations with an AI
assistant. The assistant is known for its humor and wit, providing
entertaining and amusing responses to users' questions. Here are some
examples:

Human: What's the weather like?
AI: It's raining cats and dogs, better bring an umbrella!
Human: How old are you?
AI: Age is just a number, but I'm timeless.
Human: What is the meaning of life ?
AI: 

In [59]:
llm = ChatOpenAI(model="gpt-4-turbo", api_key=openai_api_key, temperature=0.0)

In [60]:
chain = final_prompt | llm

In [62]:
display(Markdown(chain.invoke({"input": "What's the meaning of life?"}).content))

42, of course! But also, trying to find the best pizza topping. What do you think it is? Pepperoni or pineapple?

In [63]:
display(Markdown(chain.invoke({"input": "Why do we exist ?"}).content))

Ah, the million-dollar question! We might exist to enjoy life's little moments, like finding an extra fry at the bottom of the bag. Or maybe it's to ponder deep questions like this with a friendly AI!

## Open source LLMs and various applications in that

In [3]:
huggingface_api_key = os.environ["HUGGINGFACEHUB_API_TOKEN"]

In [24]:
from langchain_community.llms import HuggingFaceHub

llm = HuggingFaceHub(
    repo_id="google/flan-t5-large",
    task="text-generation",
    model_kwargs={
        "temperature": 0.1
    },
)

### Question Answering

In [31]:
from langchain_core.prompts.prompt import PromptTemplate

template = """Question: {question}

Answer: """
prompt = PromptTemplate(
    template=template,
    input_variables=['question']
)

# user question
question = "What is the capital city of Germany?"

In [32]:
chain = prompt | llm

In [33]:
chain.invoke({"question": question})

'berlin'

In [36]:
qa = [
    {'question': "What is the capital city of France?"},
    {'question': "What is the largest mammal on Earth?"},
    {'question': "Which gas is most abundant in Earth's atmosphere?"},
    {'question': "What color is a ripe banana?"}
]
res = chain.batch(qa)
print( res )

['paris', 'giraffe', 'nitrogen', 'yellow']


In [37]:
multi_template = """Answer the following questions one at a time.

Questions:
{questions}

Answers:
"""
long_prompt = PromptTemplate(template=multi_template, input_variables=["questions"])

In [43]:
from langchain.chains import LLMChain

llm_chain = LLMChain(
    prompt=long_prompt,
    llm=llm
)

In [47]:
chain = long_prompt | llm

In [50]:
chain

PromptTemplate(input_variables=['questions'], template='Answer the following questions one at a time.\n\nQuestions:\n{questions}\n\nAnswers:\n')
| HuggingFaceHub(client=<InferenceClient(model='google/flan-t5-large', timeout=None)>, repo_id='google/flan-t5-large', task='text-generation', model_kwargs={'temperature': 0.1})

In [51]:
qs_str = (
    "What is the capital city of France?\n" +
    "What is the largest mammal on Earth?\n" +
    "Which gas is most abundant in Earth's atmosphere?\n" +
		"What color is a ripe banana?\n"
)
chain.invoke({"questions": qs_str})

'Paris is the capital of France. It is also the largest city in the world. It is'